In [1]:
import tensorflow as tf 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from  tensorflow.keras import models, optimizers, regularizers

In [2]:
train_dir = '/kaggle/input/cnn-data-sources/cats_and_dogs/train'
test_dir = '/kaggle/input/cnn-data-sources/cats_and_dogs/test'
validation_dir = '/kaggle/input/cnn-data-sources/cats_and_dogs/validation'

## Tratamiento de Imagenes (Data Augmentation)

In [3]:
train_generator = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_generator = ImageDataGenerator(
    rescale = 1/255
)

test_generator = ImageDataGenerator(
    rescale = 1/255
)

In [4]:
train_data = train_generator.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 32,
    class_mode = "binary"
)

validation_data = validation_generator.flow_from_directory(
    validation_dir,
    target_size = (150, 150),
    batch_size = 32,
    class_mode = "binary"
)

test_data = test_generator.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    batch_size = 32,
    class_mode = "binary"
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Arquitectura de la Red

In [5]:
#Red neuronal
model = tf.keras.models.Sequential()

## Extracción de Características
model.add(Conv2D(32, (3,3), activation= 'relu', input_shape=(150,150,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compilamos y estamos listos para el entrenamiento
opt = tf.keras.optimizers.Adam()
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

model.summary()

2023-02-12 20:32:14.557921: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 20:32:14.558862: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 20:32:14.645352: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 20:32:14.646157: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 20:32:14.646886: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

## Entrenamiento del modelo

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "/kaggle/working/model_checkpoints_complete"
checkpoint = ModelCheckpoint(
    filepath = checkpoint_path,
    frecuency = "epoch",
    save_weights_only = False,
    monitor = "val_accuracy",
    save_best_only = True,
    verbose = 1
)

In [7]:
history = model.fit(
    train_data,
    steps_per_epoch=2000//32,
    epochs=100,
    validation_data=validation_data,
    validation_steps= 1000//32,
    callbacks=[checkpoint]
)

2023-02-12 20:32:18.376688: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2023-02-12 20:32:20.514731: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


62/62 [==============================] - 44s 565ms/step - loss: 0.6967 - accuracy: 0.5366 - val_loss: 0.7121 - val_accuracy: 0.5020

Epoch 00001: val_accuracy improved from -inf to 0.50202, saving model to /kaggle/working/model_checkpoints_complete


2023-02-12 20:33:02.516549: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/100
62/62 [==============================] - 19s 303ms/step - loss: 0.6935 - accuracy: 0.5285 - val_loss: 0.6923 - val_accuracy: 0.5010

Epoch 00002: val_accuracy did not improve from 0.50202
Epoch 3/100
62/62 [==============================] - 19s 310ms/step - loss: 0.6855 - accuracy: 0.5254 - val_loss: 0.6566 - val_accuracy: 0.5766

Epoch 00003: val_accuracy improved from 0.50202 to 0.57661, saving model to /kaggle/working/model_checkpoints_complete
Epoch 4/100
62/62 [==============================] - 19s 312ms/step - loss: 0.6883 - accuracy: 0.5401 - val_loss: 0.6768 - val_accuracy: 0.5595

Epoch 00004: val_accuracy did not improve from 0.57661
Epoch 5/100
62/62 [==============================] - 19s 304ms/step - loss: 0.6889 - accuracy: 0.5147 - val_loss: 0.6858 - val_accuracy: 0.5504

Epoch 00005: val_accuracy did not improve from 0.57661
Epoch 6/100
62/62 [==============================] - 19s 314ms/step - loss: 0.6861 - accuracy: 0.5198 - val_loss: 0.6802 - val_accuracy:

In [8]:
model.load_weights(checkpoint_path)
model.save("modelF_PG.h5")

2023-02-12 21:08:56.240181: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open /kaggle/working/model_checkpoints_complete: Failed precondition: /kaggle/working/model_checkpoints_complete; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [9]:
model.evaluate(test_data)

32/32 [==============================] - 9s 291ms/step - loss: 0.4570 - accuracy: 0.7940


[0.4569726288318634, 0.7940000295639038]